In [1]:
from v4vapp_backend_v2.database.db import MongoDBClient
from v4vapp_backend_v2.config.setup import InternalConfig
from pymongo import AsyncMongoClient
import os
import pandas as pd
import matplotlib.pyplot as plt

from v4vapp_backend_v2.actions.tracked_models import TrackedBaseModel
from v4vapp_backend_v2.actions.tracked_all import tracked_any_filter

print("Current working directory:", os.getcwd())


project_root = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2"
if os.getcwd() != project_root:
    os.chdir(project_root)
print("Current working directory:", os.getcwd())

CONFIG = InternalConfig(config_filename="config.yaml").config

old_client = MongoDBClient(
    db_conn=CONFIG.dbs_config.default_connection,
    db_name=CONFIG.dbs_config.default_name,
    db_user=CONFIG.dbs_config.default_user,
)

client = AsyncMongoClient(old_client.uri, tz_aware=True)
db = client[CONFIG.dbs_config.default_name]
rates_collection = db["rates"]
hive_ops_collection = db["hive_ops"]


2025-05-27T08:43:22+0300.472 INFO     db                        160 : 📁 backend Initializing MongoDBClient local_connection


Current working directory: /Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2/src/jupyter
Current working directory: /Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2
Config file found: config/config.yaml


In [13]:
from pprint import pprint


await TrackedBaseModel.update_quote()
cursor = hive_ops_collection.find(
    {
        "conv": {"$exists": True},
    },
    {"_id": 0},
)
async for doc in cursor:
    try:
        op = tracked_any_filter(doc)
        pprint(f"{abs(op.conv.fetch_date - op.timestamp)}")
    except ValueError:
        pass

2025-05-27T09:19:21+0300.209 INFO     crypto_prices             340 : Quotes fetched successfully in 2.1136 seconds


'2:05:23.232000'
'2:05:14.232000'
'2:02:01.400000'
'1:08:21.511000'
'1:08:03.511000'
'1:04:48.511000'
'0:44:14.707000'
'0:44:05.707000'
'0:35:11.707000'
'0:21:59.982000'
'0:08:43.377000'
'0:08:25.377000'
'0:00:42.293000'
'17:10:26.140000'
'16:08:07.976000'
'14:35:04.812000'
'14:34:55.812000'
'14:31:34.812000'
'14:26:04.812000'
'14:25:55.812000'
'14:25:34.812000'
'14:23:48.163000'
'14:23:15.163000'
'13:30:42.070000'
'13:25:51.070000'
'13:17:12.070000'
'13:16:42.070000'
'0:01:48.917000'
'0:01:54.917000'
'0:02:24.917000'
'0:00:02.088000'
'0:00:57.912000'
'0:00:57.912000'
'0:01:09.912000'
'0:01:30.912000'
'0:01:27.095000'
'0:02:00.095000'
'0:01:59.814000'
'0:02:17.814000'
'0:00:01.689000'
'0:01:07.311000'
'0:00:06.554000'
'0:00:17.446000'
'0:00:32.446000'
'0:01:26.446000'
'0:00:03.703000'
'0:00:08.297000'
'0:00:32.297000'
'0:01:47.297000'
'0:00:04.264000'
'0:00:16.736000'
'0:00:04.056000'
'0:00:00.363000'
'0:00:21.363000'
'0:00:21.363000'
'0:00:03.971000'
'0:00:05.029000'
'0:00:35.029000'
